In [8]:
%cd image_generation
!../blender/blender --background --python render_images.py -- --num_images 10 --use_gpu 1 --min_objects 2 --max_objects 3
%cd ..

/home/kevin/clevr-dataset-gen/image_generation
found bundled python: /home/kevin/clevr-dataset-gen/blender/2.78/python
read blend: data/base_scene.blend
convertViewVec: called in an invalid context
convertViewVec: called in an invalid context
0.26374387451062764 0.4 left
BROKEN MARGIN!
convertViewVec: called in an invalid context
Fra:1 Mem:16.78M (0.00M, Peak 16.78M) | Time:00:00.00 | Preparing Scene data
Fra:1 Mem:25.64M (0.00M, Peak 27.79M) | Time:00:00.02 | Preparing Scene data
Fra:1 Mem:25.64M (0.00M, Peak 27.79M) | Time:00:00.02 | Creating Shadowbuffers
Fra:1 Mem:25.64M (0.00M, Peak 27.79M) | Time:00:00.02 | Raytree.. preparing
Fra:1 Mem:35.38M (0.00M, Peak 35.38M) | Time:00:00.03 | Raytree.. building
Fra:1 Mem:34.85M (0.00M, Peak 50.06M) | Time:00:00.18 | Raytree finished
Fra:1 Mem:34.85M (0.00M, Peak 50.06M) | Time:00:00.18 | Creating Environment maps
Fra:1 Mem:34.85M (0.00M, Peak 50.06M) | Time:00:00.18 | Caching Point Densities
Fra:1 Mem:34.85M (0.00M, Peak 50.06M) | Time:00:0